In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder, TargetEncoder
# import optuna.integration.lightgbm as lgb
from autogluon.tabular import TabularDataset, TabularPredictor
import category_encoders as ce
import math

# pandasの行を省略しない
pd.set_option('display.max_columns', None)

In [65]:
train_df = pd.read_csv('C:/python/signate/data/train.csv', index_col=0)
test_df = pd.read_csv('C:/python/signate/data/test.csv', index_col=0)
sample_df = pd.read_csv('C:/python/signate/data/sample_submission.csv', header=None, index_col=0)

In [66]:
def cleansing(df):
    df['created_at'] = pd.to_datetime(df['created_at'])
    # df['cos_day'] = df['created_at'].dt.dayofyear
    # df['cos_day'] = df['cos_day'].apply(lambda x: np.cos(math.radians(90 - (x/365)*365)))
    # df['sin_day'] = df['created_at'].dt.dayofyear
    # df['sin_day'] = df['sin_day'].apply(lambda x: np.sin(math.radians(90 - (x/365)*365)))  
    df['year'] = df['created_at'].dt.year
    df['month'] = df['created_at'].dt.month
    # df['weekday'] = df['created_at'].dt.weekday
    # df['day'] = df.created_at.dt.day
    # df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    # df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df.drop('created_at', axis=1, inplace=True)
    df['steward'].fillna('0', inplace=True)
    df['guards'].fillna('0', inplace=True)
    df['problems'].fillna('NoProblem', inplace=True)
    df['spc_genus'] = df['spc_latin'].str.split(' ').str[0]
    df.drop('spc_latin', axis=1, inplace=True)


    # df['curb_loc'] = df['curb_loc'].map({'OnCurb':3, 'OffsetFromCurb':1})
    # df['sidewalk'] = df['sidewalk'].map({'NoDamage':1, 'Damage':3})
    # df['guards'] = df['guards'].map({'Helpful':1, 'Harmful':3, '0':2, 'unsure':2})
    # df['status_point'] = df['curb_loc'] * df['sidewalk'] * df['guards']
    return df

train_clean = cleansing(train_df)
test_clean = cleansing(test_df)
train_clean.head()

,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,spc_common,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist,year,month,spc_genus
0,14,OnCurb,1,0,0,Damage,Volunteer,NoProblem,English oak,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23,2015,6,Quercus
1,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,NoProblem,crimson king maple,BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15,2016,9,Acer
2,26,OnCurb,2,0,0,NoDamage,Volunteer,StonesBranchLights,English oak,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51,2015,9,Quercus
3,15,OnCurb,0,0,0,Damage,NYC Parks Staff,NoProblem,honeylocust,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51,2016,5,Gleditsia
4,23,OnCurb,1,0,0,NoDamage,Volunteer,Stones,London planetree,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9,2016,6,Platanus


In [67]:
'''
# 種目ごとの平均直径とその差を特徴量にする
test_clean['health'] = 0
all_df = pd.concat([train_clean, test_clean], axis=0)   
tmp_df = all_df[['tree_dbh', 'spc_common', ]]
dbh_mean_common = (pd.DataFrame(tmp_df.groupby('spc_common').mean()['tree_dbh']))
dbh_mean_common.rename(columns={'tree_dbh': 'dbh_mean_common'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_common, on='spc_common', how='left')
all_df['dbh_diff_common'] = all_df['tree_dbh'] - all_df['dbh_mean_common']

tmp_df = all_df[['tree_dbh', 'spc_genus', ]]
dbh_mean_genus = (pd.DataFrame(tmp_df.groupby('spc_genus').mean()['tree_dbh']))
dbh_mean_genus.rename(columns={'tree_dbh': 'dbh_mean_genus'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_genus, on='spc_genus', how='left')
all_df['dbh_diff_genus'] = all_df['tree_dbh'] - all_df['dbh_mean_genus']

train_clean = all_df.iloc[:len(train_clean)]
test_clean = all_df.iloc[len(train_clean):]
train_clean = train_clean.drop(['dbh_mean_common', 'tree_dbh'], axis=1, )
test_clean = test_clean.drop(['health', 'dbh_mean_common', 'tree_dbh'], axis=1, )
test_clean.head()
'''

"\n# 種目ごとの平均直径とその差を特徴量にする\ntest_clean['health'] = 0\nall_df = pd.concat([train_clean, test_clean], axis=0)   \ntmp_df = all_df[['tree_dbh', 'spc_common', ]]\ndbh_mean_common = (pd.DataFrame(tmp_df.groupby('spc_common').mean()['tree_dbh']))\ndbh_mean_common.rename(columns={'tree_dbh': 'dbh_mean_common'}, inplace=True)\nall_df = pd.merge(all_df, dbh_mean_common, on='spc_common', how='left')\nall_df['dbh_diff_common'] = all_df['tree_dbh'] - all_df['dbh_mean_common']\n\ntmp_df = all_df[['tree_dbh', 'spc_genus', ]]\ndbh_mean_genus = (pd.DataFrame(tmp_df.groupby('spc_genus').mean()['tree_dbh']))\ndbh_mean_genus.rename(columns={'tree_dbh': 'dbh_mean_genus'}, inplace=True)\nall_df = pd.merge(all_df, dbh_mean_genus, on='spc_genus', how='left')\nall_df['dbh_diff_genus'] = all_df['tree_dbh'] - all_df['dbh_mean_genus']\n\ntrain_clean = all_df.iloc[:len(train_clean)]\ntest_clean = all_df.iloc[len(train_clean):]\ntrain_clean = train_clean.drop(['dbh_mean_common', 'tree_dbh'], axis=1, )\ntest_clean =

In [68]:
# カウントエンコーディングパート
# すべてのカテゴリカル変数
all_categorical_cols = ['curb_loc', 'steward', 'guards', 'sidewalk',
    'user_type', 'problems', 'spc_common', 'nta', 'nta_name', 'borocode',
    'boro_ct', 'boroname', 'zip_city', 'cb_num', 'st_senate', 'st_assem',
    'cncldist', 'year', 'month',  'spc_genus', ]

# 一度yを分離する
train_y = train_clean['health']
train_X = train_clean.drop('health', axis=1)

# mergeしてfit, そのあとtransform
all_df = pd.concat([train_X, test_clean], axis=0)

# count encoding
cencoder = ce.CountEncoder(cols=all_categorical_cols)
cencoder.fit(all_df)

# count encodingを適用
train_clean_drop = cencoder.transform(train_X)
test_clean_drop = cencoder.transform(test_clean)

# yを再結合
train_clean_drop = pd.concat([train_clean_drop, train_y], axis=1)

In [74]:
# train_df, valid_df = train_test_split(train_clean_drop, test_size=0.2, random_state=42, stratify=train_clean['health'])
train_df = train_clean_drop
dir_default = 'C:/python/signate/data/autogluon/'
predictor = TabularPredictor(label='health', path=dir_default, eval_metric='f1_macro').fit(train_data=train_df)

No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for initial prototyping.
Beginning AutoGluon training ...
AutoGluon will save models to "C:/python/signate/data/autogluon/"


=================== System Info ===================
AutoGluon Version:  1.0.0
Python Version:     3.10.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
CPU Count:          8
Memory Avail:       1.42 GB / 7.85 GB (18.1%)
Disk Space Avail:   212.41 GB / 475.69 GB (44.7%)
Train Data Rows:    19984
Train Data Columns: 21
Label Column:       health
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == int, but few unique label-values observed).
	3 unique label values:  [1, 2, 0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Problem Type:       multiclass
Preprocessing data ...
Train Data Class Count: 3
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1451.42 MB
	Train Data (Orig

In [77]:
y_pred = predictor.predict(test_clean_drop)
sample_df[1] = y_pred
sample_df.to_csv('C:/python/signate/data/autogluon.csv', header=None)

In [70]:
valid_y = valid_df['health']
valid_df.drop('health', axis=1, inplace=True)
y_pred = predictor.predict(valid_df)
score = f1_score(valid_y, y_pred, average='macro')
print(f'f1_score: {score:.4f}')

f1_score: 0.3405


In [75]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                  model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0   WeightedEnsemble_L2   0.366965    f1_macro       0.329685  24.469586                0.002990           3.585828            2       True         14
1       NeuralNetFastAI   0.340350    f1_macro       0.050864  20.864809                0.050864          20.864809            1       True          3
2        KNeighborsUnif   0.333134    f1_macro       0.275831   0.018949                0.275831           0.018949            1       True          1
3        KNeighborsDist   0.327425    f1_macro       0.142621   0.027923                0.142621           0.027923            1       True          2
4               XGBoost   0.305437    f1_macro       0.089760   7.979094                0.089760           7.979094            1       True         11
5      RandomForestGini   0.2992

c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\autogluon\core\utils\plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [73]:
feature_importance = predictor.feature_importance(train_df)
feature_importance = feature_importance.sort_values(by='importance', ascending=False)
feature_importance

These features in provided data are not utilized by the predictor and will be ignored: ['nta_name', 'boroname']
Computing feature importance via permutation shuffling for 19 features using 5000 rows with 5 shuffle sets...
	89.25s	= Expected runtime (17.85s per shuffle set)
	43.32s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
tree_dbh,0.103443,0.011060,1.544459e-05,5,0.126215,0.080671
boro_ct,0.088029,0.009525,1.619399e-05,5,0.107641,0.068417
spc_common,0.087082,0.014203,8.199706e-05,5,0.116327,0.057837
nta,0.067274,0.004654,2.732187e-06,5,0.076857,0.057690
st_assem,0.064464,0.004840,3.784412e-06,5,0.074430,0.054499
month,0.062583,0.009903,7.278448e-05,5,0.082974,0.042193
problems,0.061570,0.008490,4.229973e-05,5,0.079050,0.044090
cb_num,0.060440,0.009493,7.069079e-05,5,0.079987,0.040894
spc_genus,0.057772,0.010977,1.491629e-04,5,0.080375,0.035170
cncldist,0.054976,0.001613,8.892704e-08,5,0.058298,0.051654


In [21]:
# 種目ごとの平均直径とその差を特徴量にする
test_clean['health'] = 0
all_df = pd.concat([train_clean, test_clean], axis=0)   
tmp_df = all_df[['tree_dbh', 'spc_common', ]]
dbh_mean_common = (pd.DataFrame(tmp_df.groupby('spc_common').mean()['tree_dbh']))
dbh_mean_common.rename(columns={'tree_dbh': 'dbh_mean_common'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_common, on='spc_common', how='left')
all_df['dbh_diff_common'] = all_df['tree_dbh'] - all_df['dbh_mean_common']
'''
tmp_df = all_df[['tree_dbh', 'spc_genus', ]]
dbh_mean_genus = (pd.DataFrame(tmp_df.groupby('spc_genus').mean()['tree_dbh']))
dbh_mean_genus.rename(columns={'tree_dbh': 'dbh_mean_genus'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_genus, on='spc_genus', how='left')
all_df['dbh_diff_genus'] = all_df['tree_dbh'] - all_df['dbh_mean_genus']
'''
train_clean = all_df.iloc[:len(train_clean)]
test_clean = all_df.iloc[len(train_clean):]
test_clean.drop('health', axis=1, inplace=True)
test_clean.head()


SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
'''
# trainとtestに含まれるproblemは全く同じ
# problemのonehotカラムを用意
problem_list = ['BranchLights', 'BranchOther', 'MetalGrates', 'RootOther', 'Stones', 'Sneakers', 'TrunkLights',
                 'TrunkOther', 'WiresRope', 'NoProblem']	

for problem in problem_list:
    train_clean.loc[:, problem] = 0
    test_clean.loc[:, problem] = 0

train_clean['problem_count'] = 0
test_clean['problem_count'] = 0

# problemlistにあったらonehotする
for i in train_clean.index:
    p_count = 0
    for problem in problem_list:
        if(problem in train_clean.loc[i, 'problems']):
            train_clean.loc[i, problem] = 1
            if(problem != 'NoProblem'):
                p_count+=1
    train_clean.loc[i, 'problem_count'] = p_count

for i in test_clean.index:
    p_count = 0
    for problem in problem_list:
        if(problem in test_clean.loc[i, 'problems']):
            test_clean.loc[i, problem] = 1
            if(problem != 'NoProblem'):
                p_count+=1
    test_clean.loc[i, 'problem_count'] = p_count

test_clean.head()
'''

"\n# trainとtestに含まれるproblemは全く同じ\n# problemのonehotカラムを用意\nproblem_list = ['BranchLights', 'BranchOther', 'MetalGrates', 'RootOther', 'Stones', 'Sneakers', 'TrunkLights',\n                 'TrunkOther', 'WiresRope', 'NoProblem']\t\n\nfor problem in problem_list:\n    train_clean.loc[:, problem] = 0\n    test_clean.loc[:, problem] = 0\n\ntrain_clean['problem_count'] = 0\ntest_clean['problem_count'] = 0\n\n# problemlistにあったらonehotする\nfor i in train_clean.index:\n    p_count = 0\n    for problem in problem_list:\n        if(problem in train_clean.loc[i, 'problems']):\n            train_clean.loc[i, problem] = 1\n            if(problem != 'NoProblem'):\n                p_count+=1\n    train_clean.loc[i, 'problem_count'] = p_count\n\nfor i in test_clean.index:\n    p_count = 0\n    for problem in problem_list:\n        if(problem in test_clean.loc[i, 'problems']):\n            test_clean.loc[i, problem] = 1\n            if(problem != 'NoProblem'):\n                p_count+=1\n    test_clean.

## やりたい実験
- あらゆるエンコーディング方法の比較 ok
- 地区ごとの予測モデルの実装 
- 365日で一周するsin, cos 意味無し


In [ ]:
train_clean.columns

Index(['tree_dbh', 'curb_loc', 'health', 'steward', 'guards', 'sidewalk',
       'user_type', 'problems', 'spc_common', 'nta', 'nta_name', 'borocode',
       'boro_ct', 'boroname', 'zip_city', 'cb_num', 'st_senate', 'st_assem',
       'cncldist', 'year', 'month', 'spc_genus', 'dbh_mean_common',
       'dbh_diff_common'],
      dtype='object')

In [ ]:
# エンコーディングパート

# すべてのカテゴリカル変数
all_categorical_cols = ['curb_loc', 'steward', 'guards', 'sidewalk',
       'user_type', 'problems', 'spc_common', 'nta', 'nta_name', 'borocode',
       'boro_ct', 'boroname', 'zip_city', 'cb_num', 'st_senate', 'st_assem',
       'cncldist', 'year', 'month',  'spc_genus', ]

# 

# 落とすカラム
drop_cols = ['nta_name', 'boroname', 'boro_ct',
              'nta', 'borocode', 'zip_city', 'st_senate',
              'st_assem', 'cncldist', 'dbh_mean_common', 'tree_dbh',
              ]#'',
# カウントエンコーディング
ce_columns = ['curb_loc',  'steward', 'guards', 'sidewalk',
       'user_type',  'spc_common', 'spc_genus', 'problems', 
       'year', 'month'
]
# 'nta',
#  
# 

# ターゲットエンコーディング
te_columns = []
# ラベルエンコーディング
le_columns = ['cb_num']
encoding_cals = list(drop_cols + ce_columns + te_columns + drop_cols)
if(encoding_cals != all_categorical_cols):
    print('error! categorical is not same')
    print('足りないカラム:', set(all_categorical_cols)-set(encoding_cals))

error! categorical is not same
足りないカラム: {'cb_num'}


In [ ]:
# ターゲットエンコーディングパート
if(len(te_columns) > 0):
    # onehot正解ラベルの作成
    for i in range(3):
        train_clean['onehot' + str(i)] = 0
        train_clean['onehot' + str(i)] = train_clean['onehot' + str(i)].mask(train_clean['health'] == i, 1)

    # ターゲットエンコーディングのカラムを3つずつに分ける
    te_columns_list = []

    for te_column in te_columns:
        tmp_list = []
        for i in range(3):
            train_clean[te_column + '-te' + str(i)] = train_clean[te_column]
            test_clean[te_column + '-te' + str(i)] = test_clean[te_column]
            tmp_list.append(te_column + '-te' + str(i))
        te_columns_list.append(tmp_list)
        train_clean.drop(te_column, axis=1, inplace=True)
        test_clean.drop(te_column, axis=1, inplace=True)

    te_columns_list = np.array(te_columns_list)

    # ターゲットエンコーディング
    for i in range(3):
        te_target = te_columns_list[:, i]
        target_enc = TargetEncoder(target_type='binary', random_state=42)
        
        train_clean[te_target] = target_enc.fit_transform(train_clean[te_target], train_clean['onehot' + str(i)])
        # テストデータにtransform
        test_clean[te_target]  = target_enc.transform(test_clean[te_target])

    # onehot正解ラベルを削除
    for i in range(3):
        dropcol = 'onehot' + str(i)
        train_clean.drop(dropcol, axis=1, inplace=True)


# 目的変数を指定
y = train_clean['health']

# 使わないcolumnを削除
train_clean_drop = train_clean.drop(['health'], axis=1)
train_clean_drop = train_clean_drop.drop(drop_cols, axis=1)
test_clean_drop = test_clean.drop(drop_cols,axis=1)

# ラベルエンコーディングパート
le = LabelEncoder()
for column in le_columns:
    train_clean_drop[column] = le.fit_transform(train_clean_drop[column])
    test_clean_drop[column] = le.transform(test_clean_drop[column])

# カテゴリカル変数を指定
for col in le_columns:
    train_clean_drop[col] = train_clean_drop[col].astype('category')
    test_clean_drop[col] = test_clean_drop[col].astype('category')  

# カウントエンコーディングパート
# mergeしてfit, そのあとtransform
all_df = pd.concat([train_clean_drop, test_clean_drop], axis=0)

# count encoding
cencoder = ce.CountEncoder(cols=ce_columns)
cencoder.fit(all_df)

# count encodingを適用
train_clean_drop = cencoder.transform(train_clean_drop)
test_clean_drop = cencoder.transform(test_clean_drop)

In [ ]:
train_clean_drop.head()

,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,cb_num,year,month,spc_genus,dbh_diff_common
0,37270,29409,29510,18509,22298,24288,2579,52,30331,3921,7853,1.653354
1,37270,2068,7366,21177,22298,24288,254,18,9355,6054,3613,-5.157480
2,37270,29409,29510,21177,22298,1529,2579,58,30331,6054,7853,13.653354
3,37270,29409,29510,18509,6031,24288,2104,58,9355,1113,2104,3.515684
4,37270,29409,29510,21177,22298,4455,4339,9,9355,3921,4339,3.567181


In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
valid_scores = []
models = []

for fold, (train_idx, valid_idx) in enumerate(kf.split(train_clean_drop, y)):
    print(f'fold{fold} start')
    X_train, y_train = train_clean_drop.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = train_clean_drop.iloc[valid_idx], y.iloc[valid_idx]

    lgb_train = lgb.Dataset(X_train, y_train, weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'))
    lgb_eval = lgb.Dataset(X_valid, y_valid, weight=np.ones(len(X_valid)).astype('float32'))

    '''
    params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'seed': 0,
 'feature_pre_filter': False,
 'lambda_l1': 0.0015923361719036968,
 'lambda_l2': 0.004786876640032096,
 'num_leaves': 252,
 'feature_fraction': 0.8,
 'bagging_fraction': 0.9931160872840541,
 'bagging_freq': 7,
 'min_child_samples': 5,
 'num_iterations': 1000}
 '''
    verbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される
    params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'seed': 0,
 'num_iterations': 1000}
    
    model = lgb.train(params,
                      lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=1000,
                       callbacks=[lgb.early_stopping(stopping_rounds=50, 
                                verbose=True), # early_stopping用コールバック関数
                           lgb.log_evaluation(verbose_eval)], # コマンドライン出力用コールバック関数))
                        # feval=f1,
                        # categorical_feature=le_columns,
                    )

    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_max = np.argmax(y_pred, axis=1)
    score = f1_score(y_valid, y_pred_max, average='macro')
    print(f'fold{fold} f1_score: {score:.4f}')
    valid_scores.append(score)
    models.append(model)
print(f'CV: {np.mean(valid_scores):.4f}')


fold0 start
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002021 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 546
[LightGBM] [Info] Number of data points in the train set: 15987, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 50 rounds


c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[518]	training's multi_logloss: 0.225401	valid_1's multi_logloss: 0.795617
fold0 f1_score: 0.3464
fold1 start
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000551 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 545
[LightGBM] [Info] Number of data points in the train set: 15987, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 50 rounds


c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[588]	training's multi_logloss: 0.205717	valid_1's multi_logloss: 0.784046
fold1 f1_score: 0.3574
fold2 start
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000584 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 544
[LightGBM] [Info] Number of data points in the train set: 15987, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 50 rounds


...

Early stopping, best iteration is:
[529]	training's multi_logloss: 0.224522	valid_1's multi_logloss: 0.794667
fold2 f1_score: 0.3495
fold3 start
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001141 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 546
[LightGBM] [Info] Number of data points in the train set: 15987, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 50 rounds


c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[473]	training's multi_logloss: 0.245006	valid_1's multi_logloss: 0.811447
fold3 f1_score: 0.3489
fold4 start
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001560 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 546
[LightGBM] [Info] Number of data points in the train set: 15988, number of used features: 12
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
Training until validation scores don't improve for 50 rounds


c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


Early stopping, best iteration is:
[514]	training's multi_logloss: 0.236332	valid_1's multi_logloss: 0.764604
fold4 f1_score: 0.3596
CV: 0.3524


In [ ]:
# 全データを利用して学習
'''
lgb_train = lgb.Dataset(train_clean_drop, y, weight=compute_sample_weight(class_weight='balanced', y=y).astype('float32'))
verbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される
params = {'objective': 'multiclass',
'metric': 'multi_logloss',
'num_class': 3,
'seed': 0
}

model = lgb.train(params,
                    lgb_train,
                    num_boost_round=520,
                )
'''

"\nlgb_train = lgb.Dataset(train_clean_drop, y, weight=compute_sample_weight(class_weight='balanced', y=y).astype('float32'))\nverbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される\nparams = {'objective': 'multiclass',\n'metric': 'multi_logloss',\n'num_class': 3,\n'seed': 0\n}\n\nmodel = lgb.train(params,\n                    lgb_train,\n                    num_boost_round=520,\n                )\n"

In [ ]:
y_pred = model.predict(test_clean_drop)
y_pred_max = np.argmax(y_pred, axis=1) # 最尤と判断したクラスの値にする
sample_df[1] = y_pred_max
# sample_df.to_csv('C:/python/signate/data/BEST2-nta-commondiff-dbhDrop.csv', header=None)

In [ ]:
import pandas as pd

importance = model.feature_importance()
feature_names = model.feature_name()

df_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
# importtane順に並び替え
df_importance = df_importance.sort_values('Importance', ascending=False)
df_importance


,Feature,Importance
11,dbh_diff_common,13296
6,spc_common,5746
9,month,5641
7,cb_num,5249
5,problems,4663
10,spc_genus,4096
4,user_type,2208
3,sidewalk,1380
2,guards,1296
1,steward,1166
